# Identificação das pessoas físicas associadas a cada processo gerando novo dataset.

Produtos:<br>
05 - Lista de Pessoas Fisicas ProInfo.xlsx<br>

Essa lista contem o nome de todas as pessoas físicas registradas como gestores no sistema ProInfo.

In [ ]:
%conda install tabulate

In [ ]:
%conda install openpyxl

In [ ]:
# %conda install regex

In [1]:
import pandas as pd
import regex as re
import os

if os.name == 'nt':  # Windows
    path_produtos = 'Dados\\Produtos\\'
    path_originais = 'Dados\\Coletados\\'
    path_ajustados = 'Dados\\Ajustados\\'
else:  # Ubuntu
    path_produtos = 'Dados/Produtos/'
    path_originais = 'Dados/Coletados/'
    path_ajustados = 'Dados/Ajustados/'

# Extração do Nome das Pessoas Físicas cadastradas no campo "Envolvido Pessoa Física"

In [ ]:
# Carrega DataSet gerado em "01 - Tratamento Inicial dos DataSets"
df_proinfo = pd.read_excel(path_produtos+'01 - ProInfo - Lista Consolidada v1_0.xlsx', index_col=0)

In [ ]:
df_proinfo.columns

In [ ]:
# Verifica como as informações estão estruturadas no campo "Envolvido Pessoa Física "
print(df_proinfo['Envolvido Pessoa Física'][-10:].to_markdown())

In [ ]:
# Identifica os tipos de pessoas fisicas existentes no campo "Envolvido Pessoa Física"
separadores = []
for ind in df_proinfo.index:
    palavras = str(df_proinfo['Envolvido Pessoa Física'][ind]).split(' ')
    separadores.extend([palavra for palavra in palavras if ':' in palavra])

pd.DataFrame(separadores).value_counts()


In [ ]:
# Gera um DataFrame que relaciona o nome e o tipo de cada gestor com o Protocolo.
import pandas as pd
import re

# Função para extrair nomes e tipo (Gestor ou Secretário) usando expressões regulares
def extrair_nomes_tipos(registro):
    padrao = re.compile(r'(Gestor|PF|Secretário|Responsável|Advogado|Geral|Estado|Procuradoria|Juiz(a)|Conselheiro|Promotor|Remetente):\s*([^,]+)')
    matches = padrao.findall(registro)
    return matches

def SeparaTiposPessoaFísica(df):

    # Aplicar a função à coluna 'Gestores' para criar novas colunas 'Nome' e 'Tipo'
    df['Nomes_Tipos'] = df['Envolvido Pessoa Física'].apply(extrair_nomes_tipos)

    # Explodir a lista de tuplas para várias linhas
    df_explodido = df.explode('Nomes_Tipos')

    # Dividir a tupla em duas colunas 'Nome' e 'Tipo'
    df_explodido[['Tipo','vazio', 'Nome']] = pd.DataFrame(df_explodido['Nomes_Tipos'].tolist(), index=df_explodido.index)

    # Limpar espaços extras nos nomes
    df_explodido['Nome'] = df_explodido['Nome'].str.strip()

    # Selecionar colunas desejadas no novo DataFrame
    return df_explodido[['Protocolo', 'Nome', 'Tipo']]


df_pf = SeparaTiposPessoaFísica(df_proinfo[['Protocolo','Envolvido Pessoa Física']].copy())
df_pf

In [ ]:
df_pf['Tipo'].value_counts()

In [ ]:
# Existem dois tipos não esperados: "Estado" e "Geral"
print(df_pf[df_pf['Tipo']=='Estado'].head(6))
print(df_pf[df_pf['Tipo']=='Geral'].head(16))

In [ ]:
print(df_pf[df_pf['Nome']=='RUI MORAES CRUZ'])

In [ ]:
# Elimina os tipos de pessoas físicas que não são Convenentes nem Concedentes
# Os campos do tipo Geral e Estado podem ser qualquer coisa, portanto serão mantidos.
tipos_validos = ['Gestor', 'PF', 'Secretário', 'Responsável', 'Geral', 'Estado']
df_pf = df_pf[df_pf['Tipo'].isin(tipos_validos)].copy()
df_pf['Tipo'].value_counts()

In [ ]:
# Transforma todos os nomes em letras maiúsculas para facilitar comparações.
df_pf['Nome'] = [nome.upper() for nome in df_pf['Nome']]

In [ ]:
df_pf.to_excel(path_produtos+'05 - Lista de Pessoas Fisicas ProInfo v1_0.xlsx')

Arquivo ajustado no ambiente Linux